In [26]:
import matplotlib
import os

matplotlib.use("pgf")

import matplotlib.pyplot as plt

plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update(
    {
        "pgf.texsystem": "pdflatex",
        "text.usetex": True,
        "pgf.rcfonts": False,
    }
)

latexSaveDir = "../Plots/Latex"
os.makedirs(latexSaveDir, exist_ok=True)

In [27]:
import Lib.eisImport as eisImport
import Lib.eisPlot as eisPlot

In [28]:
observationsFile = "../Data/UCT AST9AH Measurements and Observations.xlsx"
eisObservations = eisImport.groupMeasurementsAndObservations(
    eisImport.readMeasurementsAndObservations(observationsFile)
)
eisImport.inferObservationTestNames(eisObservations)

In [29]:
from typing import Dict, Tuple
import pandas as pd


def plotEisTestTemperatureRanges(
    eisObservations: Dict[str, Dict[str, pd.DataFrame]],
    saveDir: str | None = None,
    fileName: str | None = None,
    figsize: Tuple[int, int] = (16, 32),
) -> plt.Figure:
    fig, ax = plt.subplots(
        2,
        1,
        sharex=True,
        figsize=figsize,
        # squeeze=True,
    )
    fig.suptitle("EIS Test Temperatures")
    ax[0].set_title("Batch A")
    ax[1].set_title("Batch B")
    ax[0].grid(True)
    ax[1].grid(True)

    for batch in eisObservations:
        axIndex = 0 if batch == "A" else 1
        for temperature in eisObservations[batch]:
            # Plot a bar chart of ranges "Battery Min Temperature" to "Battery Max Temperature"
            for row in eisObservations[batch][temperature].iterrows():
                try:
                    minTemp = float(row[1]["Battery Min Temperature"])
                    maxTemp = float(row[1]["Battery Max Temperature"])
                except ValueError:
                    continue

                ax[axIndex].barh(
                    y=row[1]["Test"],
                    width=maxTemp - minTemp,
                    left=minTemp,
                )

                if minTemp == maxTemp:
                    # Special case for when minTemp == maxTemp, plot a single point
                    ax[axIndex].scatter(
                        x=minTemp,
                        y=row[1]["Test"],
                    )

    if saveDir is not None:
        os.makedirs(saveDir, exist_ok=True)
        savePath = os.path.join(saveDir, fileName)
        fig.savefig(savePath)

    return fig

In [30]:
temperatureRangeFig = plotEisTestTemperatureRanges(
    eisObservations,
)
temperatureRangeFig.savefig(
    os.path.join(latexSaveDir, "eisTempRanges.pgf"),
)